In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
class KMeans:
  def __init__(self):
    self.k = None
    self.file_name = None
    self.cleaned_lines = None

  # Read the file in and preprocess it
  def read_file(self, file_name):
    # Read the text file line by line
    self.file_name = file_name

    with open(file_name, 'r') as file:
      lines = file.readlines()

    # Remove the tweet id and timestamp
    cleaned_lines = [line.split('|', 2)[-1] for line in lines]

    # Remove any word that starts with the symbol @
    pattern = r'@\w+\s?'
    cleaned_lines = [re.sub(pattern, '', line) for line in cleaned_lines]

    # Remove '#' symbols
    pattern = r'#'
    cleaned_lines = [re.sub(pattern, '', line) for line in cleaned_lines]

    # Remove any URL
    pattern = r'https?://\S+|www\.\S+'
    cleaned_lines = [re.sub(pattern, '', line) for line in cleaned_lines]

    # Remove punctuations
    pattern =  r'[^\w\s]'
    cleaned_lines = [re.sub(pattern, '', line) for line in cleaned_lines]

    # Remove new lines
    pattern = r'\n'
    cleaned_lines = [re.sub(pattern, '', line) for line in cleaned_lines]

    # Convert every word to lowercase
    self.cleaned_lines = [line.lower() for line in cleaned_lines]

    return self.cleaned_lines

  # Show the first 10 lines of the cleaned lines
  def show_lines(self):
    for line in self.cleaned_lines[:10]:
      print(line)

  # Calculate and return the jaccard distance of A and B
  def jaccard_distance(self, A, B):
    set_A = set(A.split())
    set_B = set(B.split())

    intersection_length = len(set_A.intersection(set_B))
    union_length = len(set_A.union(set_B))

    jaccard_distance = 1 - (intersection_length / union_length)

    return jaccard_distance

  # Randomly select k tweets indices as centers
  def initialize_centers(self, tweets, k):

    indices = np.random.choice(len(tweets), k, replace=False)
    return indices

  def assign_to_clusters(self, tweets, centers):
    square_error = []

    # Create k lists to store the clusters
    clusters = [[i] for i in centers]

    # Loop thru all tweets
    for index in range(len(tweets)):

      if index not in centers:

        # Calculate the distance from the current tweet to all k centers
        distances = [self.jaccard_distance(tweets[index], tweets[center]) for center in centers]

        # Find the index with the smallest distance
        min_index_dist = np.argmin(np.array(distances))
        # print(min_index_dist)

        # Append the index to the correct k list
        clusters[min_index_dist].append(index)

        # Calculate square of min distance from tweet to center
        min_dist = distances[min_index_dist] ** 2

        # Append the square of the distance from tweet to center
        square_error.append(min_dist)

    # Sum of Square error
    SSE = sum(square_error)
    return clusters, SSE

  # Change the centers part of the KMeans algo.
  def update_centers(self, tweets, clusters):
    centers = []

    # Loop thru all clusters
    for cluster in clusters:
      index_dist_dict = {}

      # Find the tweet within that cluster with the smallest distance
      # That tweet will be the new center for the cluster
      for i in cluster:
        total_dist = 0
        for j in cluster:
          curr_dist = self.jaccard_distance(tweets[i], tweets[j])
          total_dist += curr_dist

        index_dist_dict[i] = total_dist

      # Append the smallest key(index) and will be new center
      key, value = min(index_dist_dict.items(), key=lambda item: item[1])
      centers.append(key)

    return centers

  # The KMeans algo. loop
  def K_Means(self, tweets, k=5):
    SSE_list = []

    # Create initial centers and initial clusters
    old_centers = self.initialize_centers(tweets, k)
    old_clusters, SSE = self.assign_to_clusters(tweets, old_centers)
    #print("SSE: ", SSE)
    SSE_list.append(SSE)

    while(1):
      # Update centers
      new_centers = self.update_centers(tweets, old_clusters)

      # Update clusters
      new_clusters, SSE = self.assign_to_clusters(tweets, new_centers)
      #print("SSE: ", SSE)
      SSE_list.append(SSE)
      #print("Old_centers index:", old_centers, "New_Centers index:", new_centers)

      # Stops the K_means (return from while loop) if centers didn't change
      if set(old_centers) == set(new_centers):
        return SSE_list, new_clusters

      else:
        # Update old_centers to new_centers
        # Update old_clusters to new_clusters
        old_centers  = new_centers
        new_centers = None
        old_clusters = new_clusters
        new_clusters = None

  def show_results(self, SSE_l, c):
    k = len(c)
    print("K:", k)
    print()
    i = 0
    for cluster in c:
      i += 1
      print(f"{i}: {len(cluster)} tweets")
    print("SSE:", SSE_l[-1])

  def driver(self, tweets, k=5):
    SSE_list, c = self.K_Means(tweets, k=k)
    self.show_results(SSE_list, c)

In [3]:
# TESTING
k_means = KMeans()
tweets = k_means.read_file("everydayhealth.txt")

In [4]:
k_means.driver(tweets, k=10)

K: 10

1: 304 tweets
2: 195 tweets
3: 155 tweets
4: 890 tweets
5: 405 tweets
6: 208 tweets
7: 84 tweets
8: 386 tweets
9: 456 tweets
10: 156 tweets
SSE: 2445.0084706365005
